In [ ]:
!pip install recommenders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.0/339.0 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.2/355.2 KB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.6/133.6 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 35.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.1/310.1 KB 23.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.4/662.4 KB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━

In [ ]:
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# user filter utils to remove  (A util function for faster process)
from recommenders.datasets.split_utils import filter_k_core

In [ ]:
def random_sampling_n_percent_user(df, n):
    np.random.seed(0)
    print('Original: %d users' % df['userID'].nunique())
    n_users = int(df['userID'].nunique() * n)
    print('After: %d users' % n_users)
    return df[df['userID'].isin(np.random.choice(df['userID'].unique(), n_users, replace=False))]

# get random 50 users for demo
def random_sampling_50_users(df):
    np.random.seed(0)
    n_users = 50
    return df[
        df["userID"].isin(
            np.random.choice(df["userID"].unique(), n_users, replace=False)
        )
    ]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Preprocess rating file

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Thesis/notebooks/data/Books/Books.csv", 
                        names=["itemID","userID","rating","timestamp"])

df = df[df['rating'] >= 3]
# filter users and items have less than 30 interaction
df = filter_k_core(df, 30)
# df.to_csv("/content/drive/MyDrive/Thesis/notebooks/data/AmazonBooks/ratings.csv", header=False, index=False)

user_set, item_set = set(df['userID'].unique()), set(df['itemID'].unique())
user_map = dict()
item_map = dict()

# # mapping user, item ID, start at 1
for u, user in enumerate(user_set):
    user_map[user] = u+1
for i, item in enumerate(item_set):
    item_map[item] = i+1
    
df["userID"] = df["userID"].apply(lambda x: user_map[x])
df["itemID"] = df["itemID"].apply(lambda x: item_map[x])
df = df.sort_values(by=["userID", "timestamp"])
df.drop(columns=["timestamp", "rating"], inplace=True)
df = df[['userID', 'itemID']]
df.to_csv("/content/drive/MyDrive/Thesis/notebooks/data/AmazonBooks/Books.txt", sep="\t", header=False, index=False)


In [ ]:
df

,userID,itemID,rating,timestamp
37407492,A00463782V7TKAP9EMNL,1983404179,4.0,1515110400
32047908,A00463782V7TKAP9EMNL,1517731879,4.0,1446336000
20564376,A00463782V7TKAP9EMNL,0991280733,4.0,1419120000
27540658,A00463782V7TKAP9EMNL,1499670028,5.0,1429228800
45922465,A00463782V7TKAP9EMNL,1544098235,5.0,1513036800
...,...,...,...,...
29243101,AZZV9PDNMCOZW,1517122562,4.0,1493510400
19417538,AZZV9PDNMCOZW,0981944272,4.0,1489190400
20682922,AZZV9PDNMCOZW,0992925444,4.0,1427155200
27506294,AZZV9PDNMCOZW,1499366027,4.0,1437177600


In [ ]:
columns_titles = ["userID","itemID", "rating", "timestamp"]
df = df.reindex(columns=columns_titles)

In [ ]:
df.to_csv("/content/drive/MyDrive/Thesis/notebooks/data/AmazonBooks/ratings.csv", header=False, index=False)

In [ ]:
# save mapping for later use
with open('/content/drive/MyDrive/Thesis/notebooks/data/AmazonBooksreduced_30f/itemmap.pkl', 'wb') as item_file:
    pickle.dump(item_map, item_file, protocol=pickle.HIGHEST_PROTOCOL)

with open('/content/drive/MyDrive/Thesis/notebooks/data/AmazonBooks/usermap.pkl', 'wb') as user_file:
    pickle.dump(user_map, user_file, protocol=pickle.HIGHEST_PROTOCOL)

## Preprocess Amzon
Books Metadata


The medata can be dowloaded here: https://cseweb.ucsd.edu/~jmcauley/datasets/amazon_v2/

In [ ]:
import pandas as pd
import pickle
import json
import gzip 
import re
from ast import literal_eval

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/MyDrive/Thesis/notebooks/data/AmazonBooks/itemmap.pkl', 'rb') as handle:
    itemmap = pickle.load(handle)

with open('/content/drive/MyDrive/Thesis/notebooks/data/AmazonBooks/usermap.pkl', 'rb') as handle:
    usermap = pickle.load(handle)

In [ ]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

metadata = getDF('/content/drive/MyDrive/Thesis/notebooks/data/meta_Books.json.gz')
use_columns = ['asin', 'title', 'category', 'description', 'brand', 'price', 'imageURL']
metadata = metadata[use_columns]
# Only process data use in training
metadata = metadata[metadata['asin'].isin(itemmap)]
# metadata.to_csv('/content/drive/MyDrive/Thesis/notebooks/data/AmazonBooks/metadata.csv', index=False)

In [ ]:
metadata = metadata[use_columns]

In [ ]:
metadata.to_csv('/content/drive/MyDrive/Thesis/notebooks/data/AmazonBooks/metadata.csv', index=False)

In [ ]:
# Preprocess on the preprocessed data
df = pd.read_csv('/content/drive/MyDrive/Thesis/notebooks/data/AmazonBooks/metadata.csv',converters={'category': literal_eval, 
                                                                                                        'description': literal_eval,
                                                                                                        'imageURL': literal_eval,
                                                                                                        'imageURLHighRes': literal_eval}, encoding='utf-8')

In [ ]:
df

,asin,title,category,description,brand,price,imageURL
0,0001050230,Love's Labour's Lost: Performed by Derek Jacob...,"[Books, Literature & Fiction, Dramas & Plays]",[William Shakespeare is widely regarded as the...,Visit Amazon's William Shakespeare Page,$20.93,[]
1,0001047868,Kidnapped (HarperCollinsAudioBooks),"[Books, Literature & Fiction, Classics]",[Grade 6 Up-Kidnapped by Robert Louis Stevenso...,Visit Amazon's Robert Louis Stevenson Page,NaN,[]
2,000100039X,The Prophet,"[Books, Literature & Fiction, Poetry]",[Kahlil Gibran was among the most important Ar...,Visit Amazon's Kahlil Gibran Page,NaN,[]
3,0001048767,Othello: Complete &amp; Unabridged,"[Books, New, Used & Rental Textbooks, Humanities]",[William Shakespeare is widely regarded as the...,Visit Amazon's William Shakespeare Page,NaN,[]
4,0001384198,The Little Engine That Could,"[Books, Literature & Fiction]","[When the other engines refuse, the Little Blu...",Piper Watty,$16.04,[]
...,...,...,...,...,...,...,...
51034,B01HA4P9V8,"Love, Always and Forever: A Sergeant Joe&#39;s...","[Books, Literature & Fiction, Genre Fiction]",[],Visit Amazon's Alexis Morgan Page,NaN,[]
51035,B01HAPIUZ4,A Gun for Hire (The Reed Ferguson Mystery Seri...,"[Books, Mystery, Thriller & Suspense, Thriller...",[],Visit Amazon's Renee Pawlish Page,NaN,[]
51036,B01HB9Q7CW,The Orphan&#39;s Tale,"[Books, Literature & Fiction, Genre Fiction]",[],Visit Amazon's Pam Jenoff Page,NaN,[]
51037,B01HDUQHCE,Bare Knuckle Luke: BBW Paranormal Romance Bear...,"[Books, Literature & Fiction, Action & Adventure]",[],Visit Amazon's Kim Fox Page,NaN,[]


In [ ]:
df['description'] = df['description'].apply(lambda x: ' '.join(x))  
df['description'] = df['description'].apply(lambda x: re.sub('<[^<]+?>', '', x))  # Remove html tags
df['description'] = df['description'].apply(lambda x: re.sub(r'[^\x00-\x7F]+', ' ', x))  # Remove non-ascii characters
df['description'] = df['description'].apply(lambda x: x.strip())
# df["description"] = df["description"].apply(lambda x: " " )
link = "https://ws-na.amazon-adsystem.com/widgets/q?_encoding=UTF8&MarketPlace=US&ASIN={}&ServiceVersion=20070822&ID=AsinImage&WS=1&Format=SL250"
df["imageURL"] = df["asin"].apply(lambda x: link.format(x))
# df['description'] = df['description'].apply(lambda x: x.replace(''))  # Replace empty description with title column

# remove $ sign, replace not given price with 10
for index, row in df.iterrows():          
    try:
        row['price'] = float(row['price'].replace('$', ''))
    except:
        row['price'] = 10

In [ ]:
df = df[['asin', 'title', 'category', 'description', 'price', 'imageURL']]

In [ ]:
df

,asin,title,category,description,brand,price,imageURL
0,0001050230,Love's Labour's Lost: Performed by Derek Jacob...,"[Books, Literature & Fiction, Dramas & Plays]",William Shakespeare is widely regarded as the ...,Visit Amazon's William Shakespeare Page,20.93,https://ws-na.amazon-adsystem.com/widgets/q?_e...
1,0001047868,Kidnapped (HarperCollinsAudioBooks),"[Books, Literature & Fiction, Classics]",Grade 6 Up-Kidnapped by Robert Louis Stevenson...,Visit Amazon's Robert Louis Stevenson Page,10,https://ws-na.amazon-adsystem.com/widgets/q?_e...
2,000100039X,The Prophet,"[Books, Literature & Fiction, Poetry]",Kahlil Gibran was among the most important Ara...,Visit Amazon's Kahlil Gibran Page,10,https://ws-na.amazon-adsystem.com/widgets/q?_e...
3,0001048767,Othello: Complete &amp; Unabridged,"[Books, New, Used & Rental Textbooks, Humanities]",William Shakespeare is widely regarded as the ...,Visit Amazon's William Shakespeare Page,10,https://ws-na.amazon-adsystem.com/widgets/q?_e...
4,0001384198,The Little Engine That Could,"[Books, Literature & Fiction]","When the other engines refuse, the Little Blue...",Piper Watty,16.04,https://ws-na.amazon-adsystem.com/widgets/q?_e...
...,...,...,...,...,...,...,...
51034,B01HA4P9V8,"Love, Always and Forever: A Sergeant Joe&#39;s...","[Books, Literature & Fiction, Genre Fiction]",,Visit Amazon's Alexis Morgan Page,10,https://ws-na.amazon-adsystem.com/widgets/q?_e...
51035,B01HAPIUZ4,A Gun for Hire (The Reed Ferguson Mystery Seri...,"[Books, Mystery, Thriller & Suspense, Thriller...",,Visit Amazon's Renee Pawlish Page,10,https://ws-na.amazon-adsystem.com/widgets/q?_e...
51036,B01HB9Q7CW,The Orphan&#39;s Tale,"[Books, Literature & Fiction, Genre Fiction]",,Visit Amazon's Pam Jenoff Page,10,https://ws-na.amazon-adsystem.com/widgets/q?_e...
51037,B01HDUQHCE,Bare Knuckle Luke: BBW Paranormal Romance Bear...,"[Books, Literature & Fiction, Action & Adventure]",,Visit Amazon's Kim Fox Page,10,https://ws-na.amazon-adsystem.com/widgets/q?_e...


In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
df['imageURL']

0        https://ws-na.amazon-adsystem.com/widgets/q?_encoding=UTF8&MarketPlace=US&ASIN=0001050230&ServiceVersion=20070822&ID=AsinImage&WS=1&Format=SL250
1        https://ws-na.amazon-adsystem.com/widgets/q?_encoding=UTF8&MarketPlace=US&ASIN=0001047868&ServiceVersion=20070822&ID=AsinImage&WS=1&Format=SL250
2        https://ws-na.amazon-adsystem.com/widgets/q?_encoding=UTF8&MarketPlace=US&ASIN=000100039X&ServiceVersion=20070822&ID=AsinImage&WS=1&Format=SL250
3        https://ws-na.amazon-adsystem.com/widgets/q?_encoding=UTF8&MarketPlace=US&ASIN=0001048767&ServiceVersion=20070822&ID=AsinImage&WS=1&Format=SL250
4        https://ws-na.amazon-adsystem.com/widgets/q?_encoding=UTF8&MarketPlace=US&ASIN=0001384198&ServiceVersion=20070822&ID=AsinImage&WS=1&Format=SL250
                                                                               ...                                                                       
51034    https://ws-na.amazon-adsystem.com/widgets/q?_encoding=UTF8&MarketPl

In [ ]:
df.to_json('/content/drive/MyDrive/Thesis/notebooks/data/AmazonBooks/products.json', orient='records')

In [ ]:
with open('/content/drive/MyDrive/Thesis/notebooks/data/AmazonBooks/products.json', encoding='utf-8') as f:
    metadata = json.load(f)

In [ ]:
metadata

[{'asin': '0001050230',
  'title': "Love's Labour's Lost: Performed by Derek Jacobi, Geraldine McEwan &amp; Cast",
  'category': ['Books', 'Literature & Fiction', 'Dramas & Plays'],
  'description': 'William Shakespeare is widely regarded as the greatest playwright the world has seen. He produced an astonishing amount of work; 37 plays, 154 sonnets, and 5 poems. He died on 23rd April 1616, aged 52, and was buried in the Holy Trinity Church, Stratford.',
  'brand': "Visit Amazon's William Shakespeare Page",
  'price': 20.93,
  'imageURL': 'https://ws-na.amazon-adsystem.com/widgets/q?_encoding=UTF8&MarketPlace=US&ASIN=0001050230&ServiceVersion=20070822&ID=AsinImage&WS=1&Format=SL250'},
 {'asin': '0001047868',
  'title': 'Kidnapped (HarperCollinsAudioBooks)',
  'category': ['Books', 'Literature & Fiction', 'Classics'],
  'description': "Grade 6 Up-Kidnapped by Robert Louis Stevenson remains one of the classic coming-of-age stories for children and young adults today. After the death of hi

In [ ]:
def replace_category_amp(data):
    for i in range(len(data)):
        for j in range(len(data[i]['category'])):
            data[i]['category'][j] = data[i]['category'][j].replace('&amp;', '&')
    return data

metadata = replace_category_amp(metadata)